## Using Variational Estimates to Initialize the NUTS-HMC Sampler

In this example we show how to use the parameter estimates return by Stan's variational inference algorithm
as the initial parameter values for Stan's NUTS-HMC sampler.
By default, the sampler algorithm randomly initializes all model parameters in the range uniform\[-2, 2\].  When the true parameter value is outside of this range, starting from the ADVI estimates will speed up and improve adaptation.

### Model and data

The Stan model and data are taken from the [posteriordb package](https://github.com/stan-dev/posteriordb).

We use the [blr model](https://github.com/stan-dev/posteriordb/blob/master/posterior_database/models/stan/blr.stan),
a Bayesian standard linear regression model with noninformative priors,
and its corresponding simulated dataset [sblri.json](https://github.com/stan-dev/posteriordb/blob/master/posterior_database/data/data/sblri.json.zip),
which was simulated via script [sblr.R](https://github.com/stan-dev/posteriordb/blob/master/posterior_database/data/data-raw/sblr/sblr.R).
For conveince, we have copied the posteriordb model and data to this directory, in files `blr.stan` and `sblri.json`.

In [ ]:
import os
from cmdstanpy import CmdStanModel

stan_file = 'blr.stan' # basic linear regression
data_file = 'sblri.json' # simulated data

model = CmdStanModel(stan_file=stan_file)

print(model.code())

### Run Stan's `pathfinder` or `variational` algorithm, obtain fitted estimates

The [CmdStanModel pathfinder](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanModel.pathfinder ) method
wraps the CmdStan [pathfinder ](https://mc-stan.org/docs/cmdstan-guide/pathfinder-config.html) method.

Pathfinder locates normal approximations to the target
density along a quasi-Newton optimization path, with local covariance
estimated using the negative inverse Hessian estimates produced by the
LBFGS optimizer.  Pathfinder returns draws from the Gaussian approximation
with the lowest estimated Kullback-Leibler (KL) divergence to the true
posterior.
By default, CmdStanPy runs multi-path Pathfinder which returns an importance-resampled set of draws over the outputs of 4 independent single-path Pathfinders.
This better matches non-normal target densities and also mitigates
the problem of L-BFGS getting stuck at local optima or in saddle points on plateaus.

The method [create_inits](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanPathfinder.create_inits) returns a Python Dict containing a set of per-chain initializations for the model parameters.  Each set of initializations is a random draw from the Pathfinder sample.

In [ ]:
pathfinder_fit = model.pathfinder(data=data_file, seed=123)

Posteriordb provides reference posteriors for all models.  For the blr model, conditioned on the dataset `sblri.json`, the reference posteriors are in file [sblri-blr.json](https://github.com/stan-dev/posteriordb/blob/master/posterior_database/reference_posteriors/summary_statistics/mean/mean/sblri-blr.json)

The reference posteriors for all elements of `beta` and `sigma` are all very close to $1.0$.

The experiments reported in Figure 3 of the paper [Pathfinder: Parallel quasi-Newton variational inference](https://arxiv.org/abs/2108.03782) by Zhang et al. show that Pathfinder provides a better  estimate of the posterior, as measured by the 1-Wasserstein distance to the reference posterior, than 75 iterations of the warmup Phase I algorithm used by the NUTS-HMC sampler.
furthermore, Pathfinder is more computationally efficient, requiring fewer evaluations of the log density and gradient functions.  Therefore, using the estimates from Pathfinder to initialize the parameter values for the NUTS-HMC sampler will allow the sampler to do a better job of adapting the stepsize and metric during warmup, resulting in better performance and estimation.

In [ ]:
pathfinder_inits = pathfinder_fit.create_inits()
print(pathfinder_inits)

In [ ]:
mcmc_pathfinder_inits_fit = model.sample(
    data=data_file, inits=pathfinder_inits, iter_warmup=75, seed=12345
)

In [ ]:
mcmc_pathfinder_inits_fit.summary()

Using the default random parameter initializations, we need to run more warmup iterations. If we only run 75 warmup iterations with random inits, the result fails to estimate `sigma` correctly.  It is necessary to run the model with at least 150 warmup iterations to produce a good set of estimates.

In [ ]:
mcmc_random_inits_fit = model.sample(data=data_file, iter_warmup=75, seed=12345)

In [ ]:
mcmc_random_inits_fit.summary()

In [ ]:
print(mcmc_random_inits_fit.diagnose())

The `CmdStanModel` method `variational` runs CmdStan's ADVI algorithm.
Because this algorithm is unstable and may fail to converge, we run it with argument `require_converged` set to `False`.  We also specify a seed, to avoid instabilities as well as for reproducibility.

In [ ]:
vb_fit = model.variational(data=data_file, require_converged=False, seed=123)

The ADVI algorithm provides estimates of all model parameters.

The `variational` method returns a `CmdStanVB` object, with method `stan_variables`, which
returns the approximat posterior samples of all model parameters as a Python dictionary. 
Here, we report the approximate posterior mean.

In [ ]:
vb_mean = {var: samples.mean(axis=0) for var, samples in vb_fit.stan_variables().items()}
print(vb_mean)

In [ ]:
mcmc_vb_inits_fit = model.sample(
    data=data_file, inits=vb_mean, iter_warmup=75, seed=12345
)

In [ ]:
mcmc_vb_inits_fit.summary()

The sampler estimates match the reference posterior.

In [ ]:
print(mcmc_vb_inits_fit.diagnose())